In [1]:
from huggingface_hub import hf_hub_download
import zstandard as zstd
import io
import json
import random

In [7]:
# Укажите нужные файлы (изменяйте этот список по необходимости)
file_names = [
    "basket-01.json.zst",
    "basket-02.json.zst",
    # "basket-03.json.zst",
    # "basket-04.json.zst",
]

# Основные параметры
max_total_items = 1000  # Сколько всего записей мы хотим получить
min_description_length = 100  # Минимальная длина описания для фильтрации
max_description_length = 200

print(f"Будем загружать файлы: {file_names}")
print(f"Целевое количество записей: {max_total_items}")
print(f"Минимальная длина описания: {min_description_length} символов")

Будем загружать файлы: ['basket-01.json.zst', 'basket-02.json.zst']
Целевое количество записей: 1000
Минимальная длина описания: 100 символов


In [8]:
print("=" * 50)
print("Начинаю загрузку файлов...")

file_paths = []
for i, filename in enumerate(file_names):
    print(f"Загрузка файла {i+1}/{len(file_names)}: {filename}")
    
    try:
        file_path = hf_hub_download(
            repo_id="nyuuzyou/wb-products",  # Репозиторий с данными
            filename=filename,               # Имя файла для скачивания
            repo_type='dataset'              # Тип репозитория
        )
        file_paths.append(file_path)
        print(f"  ✓ Успешно загружен: {file_path}")
    except Exception as e:
        print(f"  ✗ Ошибка при загрузке {filename}: {e}")

print(f"Всего загружено файлов: {len(file_paths)}")

Начинаю загрузку файлов...
Загрузка файла 1/2: basket-01.json.zst
  ✓ Успешно загружен: C:\Users\Admin\.cache\huggingface\hub\datasets--nyuuzyou--wb-products\snapshots\0e3062c1f1b0f690494844c4465e220fa4ec8260\basket-01.json.zst
Загрузка файла 2/2: basket-02.json.zst
  ✓ Успешно загружен: C:\Users\Admin\.cache\huggingface\hub\datasets--nyuuzyou--wb-products\snapshots\0e3062c1f1b0f690494844c4465e220fa4ec8260\basket-02.json.zst
Всего загружено файлов: 2


In [9]:
print("=" * 50)
print("Начинаю сбор данных из всех файлов...")

all_candidates = []
buffer_multiplier = 2  # Берем в 2 раза больше данных, чем нужно, чтобы было что перемешивать
target_buffer_size = max_total_items * buffer_multiplier

print('target_buffer_size:', target_buffer_size)

for i, file_path in enumerate(file_paths):
    print(f"Чтение файла {i+1}/{len(file_paths)}: {file_names[i]}")
    
    items_in_this_file = 0
    try:
        with open(file_path, "rb") as compressed:  # Открываем как бинарный файл
            # Создаем декомпрессор для формата Zstandard
            dctx = zstd.ZstdDecompressor()
            
            # Создаем поток для чтения распакованных данных
            stream = dctx.stream_reader(compressed)
            
            # Оборачиваем поток в текстовый ридер с UTF-8 кодировкой
            text_stream = io.TextIOWrapper(stream, encoding="utf-8")
            
            # Читаем файл построчно (каждая строка - отдельный JSON объект)
            for line in text_stream:
                try:
                    obj = json.loads(line)  # Парсим JSON строку
                    
                    # Фильтруем по длине описания
                    if len(obj.get("description", "")) > min_description_length \
                        and len(obj.get("description", "")) < max_description_length:
                        all_candidates.append(obj)
                        items_in_this_file += 1
                except json.JSONDecodeError:
                    # Пропускаем некорректные JSON строки
                    continue
                
                # Если собрали достаточно данных, прерываем чтение
                if len(all_candidates) >= target_buffer_size:
                    print(f"  Достигнут буфер в {target_buffer_size} записей")
                    break
                    
        print(f"  ✓ Извлечено записей: {items_in_this_file}")
        
    except Exception as e:
        print(f"  ✗ Ошибка при чтении файла: {e}")
    
    # Если уже собрали достаточно данных, останавливаемся
    if len(all_candidates) >= target_buffer_size:
        break

print(f"\nВсего собрано кандидатов: {len(all_candidates)}")

Начинаю сбор данных из всех файлов...
target_buffer_size: 2000
Чтение файла 1/2: basket-01.json.zst
  Достигнут буфер в 2000 записей
  ✓ Извлечено записей: 2000

Всего собрано кандидатов: 2000


In [10]:
print("=" * 50)
print("Перемешиваю данные...")

# Перемешиваем все собранные записи
random.shuffle(all_candidates)

print(f"Данные перемешаны")
print(f"Беру первые {max_total_items} записей из перемешанного списка")

# Берем нужное количество записей из перемешанного списка
selected = all_candidates[:max_total_items]

print(f"Итоговый размер выборки: {len(selected)} записей")

Перемешиваю данные...
Данные перемешаны
Беру первые 1000 записей из перемешанного списка
Итоговый размер выборки: 1000 записей


In [11]:
print("=" * 50)
print("Обеспечение равномерного распределения по категориям...")

# Создаем словарь для группировки по категориям
from collections import defaultdict
categories_dict = defaultdict(list)

# Группируем записи по subj_name
for item in all_candidates:
    category = item.get('subj_name', 'Без категории')
    categories_dict[category].append(item)

print(f"Найдено категорий: {len(categories_dict)}")

# Определяем сколько записей взять из каждой категории
target_per_category = max_total_items // len(categories_dict)
print(f"Будем брать примерно по {target_per_category} записей из каждой категории")

# Собираем равномерную выборку
uniform_selected = []
for category, items in categories_dict.items():
    # Перемешиваем записи внутри категории
    random.shuffle(items)
    
    # Берем нужное количество записей из категории
    items_to_take = min(target_per_category, len(items))
    uniform_selected.extend(items[:items_to_take])
    
    print(f"  Категория '{category}': {len(items)} всего, взято {items_to_take}")

print(f"Предварительная выборка: {len(uniform_selected)} записей")

# Если не набрали достаточно, добираем из остатков
if len(uniform_selected) < max_total_items:
    print(f"\nДобираем до {max_total_items} записей...")
    
    # Собираем все оставшиеся записи
    remaining_items = []
    for category, items in categories_dict.items():
        if len(items) > target_per_category:
            remaining_items.extend(items[target_per_category:])
    
    random.shuffle(remaining_items)
    
    # Сколько нужно добрать
    need_more = max_total_items - len(uniform_selected)
    uniform_selected.extend(remaining_items[:need_more])
    
    print(f"Добрано из остатков: {min(need_more, len(remaining_items))}")

# Заменяем исходный список
all_candidates = uniform_selected
print(f"Итоговое количество кандидатов: {len(all_candidates)}")

Обеспечение равномерного распределения по категориям...
Найдено категорий: 345
Будем брать примерно по 2 записей из каждой категории
  Категория 'Футболки': 96 всего, взято 2
  Категория 'Чехлы для телефонов': 78 всего, взято 2
  Категория 'Платья': 75 всего, взято 2
  Категория 'Серьги': 20 всего, взято 2
  Категория 'Головоломки': 1 всего, взято 1
  Категория 'Брюки': 36 всего, взято 2
  Категория 'Полотенцесушители': 83 всего, взято 2
  Категория 'Шляпы': 10 всего, взято 2
  Категория 'Кресла компьютерные': 2 всего, взято 2
  Категория 'Инструменты для чистки лица': 3 всего, взято 2
  Категория 'Блузки': 43 всего, взято 2
  Категория 'Ленты измерительные': 3 всего, взято 2
  Категория 'Постельное белье': 60 всего, взято 2
  Категория 'Колье': 15 всего, взято 2
  Категория 'Игровые телефоны': 2 всего, взято 2
  Категория 'Чайники заварочные': 1 всего, взято 1
  Категория 'Супы быстрого приготовления': 2 всего, взято 2
  Категория 'Косметички': 2 всего, взято 2
  Категория 'Босоножки'

In [12]:
print("=" * 50)
print("РЕЗУЛЬТАТЫ С УЧЕТОМ НОВЫХ КРИТЕРИЕВ:")

if selected:
    # Показываем первую запись
    print("\nПервая запись в финальной выборке:")
    print("-" * 50)
    print(json.dumps(selected[0], ensure_ascii=False, indent=2))
    print("-" * 50)
    
    # Статистика
    print(f"\nОБЩАЯ СТАТИСТИКА:")
    print(f"• Всего записей: {len(selected)}")
    
    # Длина описания (мин/макс/средняя)
    desc_lengths = [len(item.get("description", "")) for item in selected]
    min_desc_len = min(desc_lengths)
    max_desc_len = max(desc_lengths)
    avg_desc_len = sum(desc_lengths) / len(selected)
    
    print(f"\n• ДЛИНА ОПИСАНИЯ:")
    print(f"  - Минимальная: {min_desc_len} символов")
    print(f"  - Максимальная: {max_desc_len} символов")
    print(f"  - Средняя: {avg_desc_len:.1f} символов")
    print(f"  - Целевой диапазон: 100-200 символов")
    
    # Проверка соответствия критериям
    out_of_range = sum(1 for length in desc_lengths if length < 100 or length > 200)
    print(f"  - Записей вне диапазона: {out_of_range} ({out_of_range/len(selected)*100:.1f}%)")
    
    # Распределение по категориям (subj_name)
    from collections import Counter
    categories = [item.get('subj_name', 'Без категории') for item in selected]
    category_counter = Counter(categories)
    
    print(f"\n• РАСПРЕДЕЛЕНИЕ ПО КАТЕГОРИЯМ (subj_name):")
    print(f"  - Всего уникальных категорий: {len(category_counter)}")
    
    # Вычисляем равномерность распределения
    counts = list(category_counter.values())
    avg_count = sum(counts) / len(counts)
    max_diff = max(counts) - min(counts)
    uniformity_score = 1 - (max_diff / avg_count) if avg_count > 0 else 0
    
    print(f"  - Среднее количество на категорию: {avg_count:.1f}")
    print(f"  - Максимальное в категории: {max(counts)}")
    print(f"  - Минимальное в категории: {min(counts)}")
    print(f"  - Оценка равномерности: {uniformity_score:.2%}")
    
    print(f"\n  Топ-10 категорий по количеству записей:")
    for category, count in category_counter.most_common(10):
        percentage = count / len(selected) * 100
        print(f"    - {category}: {count} записей ({percentage:.1f}%)")
    
    # Уникальные бренды
    brands = [item.get("brand_name", "Не указан") for item in selected]
    unique_brands = set(brands)
    
    print(f"\n• БРЕНДЫ:")
    print(f"  - Уникальных брендов: {len(unique_brands)}")
    
    brand_counter = Counter(brands)
    print(f"  - Топ-5 самых частых брендов:")
    for brand, count in brand_counter.most_common(5):
        percentage = count / len(selected) * 100
        print(f"    - {brand}: {count} записей ({percentage:.1f}%)")
    
    # Дополнительная информация о корневых категориях
    root_categories = [item.get('subj_root_name', 'Без корневой категории') for item in selected]
    root_counter = Counter(root_categories)
    
    print(f"\n• КОРНЕВЫЕ КАТЕГОРИИ (subj_root_name):")
    print(f"  - Уникальных корневых категорий: {len(root_counter)}")
    print(f"  - Распределение:")
    for root_cat, count in root_counter.most_common(5):
        percentage = count / len(selected) * 100
        print(f"    - {root_cat}: {count} записей ({percentage:.1f}%)")
    
    # Примеры из выборки (демонстрация разнообразия)
    print(f"\n• ПРИМЕРЫ ЗАПИСЕЙ (первые 5):")
    for i, item in enumerate(selected[:5]):
        desc_len = len(item.get("description", ""))
        category = item.get('subj_name', 'N/A')
        print(f"{i+1}. {item.get('brand_name', 'N/A')}")
        print(f"   Категория: {category}")
        print(f"   Название: {item.get('imt_name', 'N/A')[:60]}...")
        print(f"   Длина описания: {desc_len} символов")
        if i < 4:
            print()
    
    # Сводка по критериям отбора
    print(f"\n• СВОДКА ПО КРИТЕРИЯМ ОТБОРА:")
    print(f"  ✓ Длина описания: 100-200 символов")
    print(f"  ✓ Равномерное распределение по категориям")
    print(f"  ✓ Итоговое количество: {len(selected)} записей")
    
    # Гистограмма длин описаний (текстовая)
    print(f"\n• ГИСТОГРАММА ДЛИН ОПИСАНИЙ:")
    bins = [100, 120, 140, 160, 180, 200]
    for i in range(len(bins)-1):
        start, end = bins[i], bins[i+1]
        count = sum(1 for length in desc_lengths if start <= length < end)
        percentage = count / len(desc_lengths) * 100
        bar = "█" * int(percentage / 2)  # Один символ = 2%
        print(f"  {start:3}-{end:3} символов: {count:4} ({percentage:5.1f}%) {bar}")
    
    # Считаем записи с длиной ровно 200
    exactly_200 = sum(1 for length in desc_lengths if length == 200)
    if exactly_200 > 0:
        print(f"  Ровно 200 символов: {exactly_200} записей")
        
else:
    print("Не удалось собрать ни одной записи. Проверьте параметры фильтрации.")

РЕЗУЛЬТАТЫ С УЧЕТОМ НОВЫХ КРИТЕРИЕВ:

Первая запись в финальной выборке:
--------------------------------------------------
{
  "imt_id": 10016198,
  "nm_id": 13372527,
  "imt_name": "Футболка",
  "subj_name": "Футболки",
  "subj_root_name": "Одежда",
  "nm_colors_names": "оранжевый",
  "vendor_code": "S208оранжевый",
  "description": "Одежда ESPORT отличается лекгостью тканиЮ удобной ноской и приятная телу. Швы аккуратные,  яркие цвета, упругая и втоже время тянеться до нужной посадки.",
  "brand_name": "Esport"
}
--------------------------------------------------

ОБЩАЯ СТАТИСТИКА:
• Всего записей: 1000

• ДЛИНА ОПИСАНИЯ:
  - Минимальная: 101 символов
  - Максимальная: 199 символов
  - Средняя: 146.5 символов
  - Целевой диапазон: 100-200 символов
  - Записей вне диапазона: 0 (0.0%)

• РАСПРЕДЕЛЕНИЕ ПО КАТЕГОРИЯМ (subj_name):
  - Всего уникальных категорий: 266
  - Среднее количество на категорию: 3.8
  - Максимальное в категории: 53
  - Минимальное в категории: 1
  - Оценка равномер

In [13]:
print("=" * 50)
print("СОХРАНЕНИЕ В PARQUET И CSV ФОРМАТЫ:")

import pandas as pd
import os
from pathlib import Path

# Создаем папку для сохранения, если её нет
output_dir = os.path.join(os.getcwd(), "data", "goods")
Path(output_dir).mkdir(parents=True, exist_ok=True)
print(f"✓ Создана/проверена папка: {output_dir}")

# Преобразуем данные в DataFrame для удобной работы
print("\nПреобразую данные в DataFrame...")
try:
    df = pd.DataFrame(selected)
    print(f"✓ DataFrame создан. Размер: {df.shape[0]} строк × {df.shape[1]} столбцов")
    
    # Выводим информацию о столбцах
    print(f"\nСтолбцы в данных ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2}. {col}")
    
except Exception as e:
    print(f"✗ Ошибка при создании DataFrame: {e}")
    df = None

if df is not None:
    # 1. Сохранение в Parquet (бинарный формат, эффективный для анализа)
    print("\n" + "-" * 30)
    print("Сохранение в Parquet формат...")
    
    parquet_path = os.path.join(output_dir, "goods_data.parquet")
    try:
        # Сохраняем в Parquet с использованием snappy сжатия
        df.to_parquet(parquet_path, engine='pyarrow', compression='snappy')
        
        # Получаем размер файла
        file_size_mb = os.path.getsize(parquet_path) / (1024 * 1024)
        print(f"✓ Parquet файл сохранен: {parquet_path}")
        print(f"  Размер файла: {file_size_mb:.2f} MB")
        print(f"  Сжатие: snappy")
        print(f"  Столбцы: {len(df.columns)}")
        
    except Exception as e:
        print(f"✗ Ошибка при сохранении Parquet: {e}")
        
        # Попробуем без сжатия
        try:
            df.to_parquet(parquet_path, engine='pyarrow', compression=None)
            file_size_mb = os.path.getsize(parquet_path) / (1024 * 1024)
            print(f"✓ Parquet файл сохранен без сжатия: {file_size_mb:.2f} MB")
        except Exception as e2:
            print(f"✗ Ошибка при сохранении без сжатия: {e2}")

    # 2. Сохранение в CSV (текстовый формат, удобный для Excel)
    print("\n" + "-" * 30)
    print("Сохранение в CSV формат...")
    
    csv_path = os.path.join(output_dir, "goods_data.csv")
    try:
        # Сохраняем в CSV с кодировкой UTF-8
        df.to_csv(csv_path, index=False, encoding='utf-8-sig', sep='*' )
        
        # Получаем размер файла
        file_size_mb = os.path.getsize(csv_path) / (1024 * 1024)
        print(f"✓ CSV файл сохранен: {csv_path}")
        print(f"  Размер файла: {file_size_mb:.2f} MB")
        print(f"  Кодировка: UTF-8")
        print(f"  Разделитель: запятая")
        
        # Также создаем CSV с разделителем-табуляцией для удобства
        tsv_path = os.path.join(output_dir, "goods_data_tab.сsv")
        # df.to_csv(tsv_path, index=False, encoding='utf-8-sig', sep='\t')
        tsv_size_mb = os.path.getsize(tsv_path) / (1024 * 1024)
        print(f"✓ TSV файл сохранен: {tsv_path}")
        print(f"  Размер файла: {tsv_size_mb:.2f} MB")
        print(f"  Разделитель: табуляция")
        
    except Exception as e:
        print(f"✗ Ошибка при сохранении CSV: {e}")

    # 3. Сохранение в Excel (если нужен)
    print("\n" + "-" * 30)
    print("Сохранение в Excel формат (опционально)...")
    
    excel_path = os.path.join(output_dir, "goods_data.xlsx")
    try:
        # Используем openpyxl как движок
        df.to_excel(excel_path, index=False, engine='openpyxl')
        
        file_size_mb = os.path.getsize(excel_path) / (1024 * 1024)
        print(f"✓ Excel файл сохранен: {excel_path}")
        print(f"  Размер файла: {file_size_mb:.2f} MB")
        
    except ImportError:
        print("✗ Для сохранения в Excel требуется установить openpyxl:")
        print("  pip install openpyxl")
    except Exception as e:
        print(f"✗ Ошибка при сохранении Excel: {e}")

    # 4. Создаем README файл с информацией о данных
    print("\n" + "-" * 30)
    print("Создание README файла...")
    
    readme_path = os.path.join(output_dir, "README.md")
    try:
        with open(readme_path, "w", encoding="utf-8") as f:
            f.write("# Данные товаров\n\n")
            f.write(f"## Общая информация\n")
            f.write(f"- Дата создания: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"- Количество записей: {len(df)}\n")
            f.write(f"- Количество столбцов: {len(df.columns)}\n")
            f.write(f"- Источники файлов: {', '.join(file_names)}\n\n")
            
            f.write("## Файлы\n")
            f.write("1. `goods_data.parquet` - данные в Parquet формате (рекомендуется для анализа)\n")
            f.write("2. `goods_data.csv` - данные в CSV формате с разделителем-запятой\n")
            f.write("3. `goods_data.tsv` - данные в TSV формате с разделителем-табуляцией\n")
            f.write("4. `goods_data.xlsx` - данные в Excel формате\n\n")
            
            f.write("## Столбцы\n")
            f.write("| № | Имя столбца | Тип данных | Описание |\n")
            f.write("|---|------------|------------|----------|\n")
            
            # Определяем типы данных
            for i, col in enumerate(df.columns, 1):
                dtype = str(df[col].dtype)
                # Простое описание на основе имени столбца
                description = {
                    'imt_id': 'ID товара в системе',
                    'nm_id': 'Артикул товара',
                    'imt_name': 'Название товара',
                    'subj_name': 'Название подкатегории',
                    'subj_root_name': 'Название основной категории',
                    'nm_colors_names': 'Цвета товара',
                    'vendor_code': 'Артикул поставщика',
                    'description': 'Описание товара',
                    'brand_name': 'Название бренда'
                }.get(col, 'Неизвестное поле')
                
                f.write(f"| {i} | {col} | {dtype} | {description} |\n")
            
            f.write("\n## Статистика\n")
            f.write(f"- Средняя длина описания: {avg_desc_len:.1f} символов\n")
            f.write(f"- Уникальных брендов: {len(unique_brands)}\n")
            f.write(f"- Минимальная длина описания для отбора: {min_description_length} символов\n")
        
        print(f"✓ README файл создан: {readme_path}")
        
    except Exception as e:
        print(f"✗ Ошибка при создании README: {e}")

    # 5. Выводим сводную информацию
    print("\n" + "=" * 50)
    print("СВОДНАЯ ИНФОРМАЦИЯ О СОХРАНЕННЫХ ДАННЫХ:")
    print(f"Папка с данными: {output_dir}")
    print(f"Файлы сохранены в следующие форматы:")
    
    # Проверяем какие файлы создались
    files_created = []
    for file_name in ["goods_data.parquet", "goods_data.csv", "goods_data.tsv", 
                      "goods_data.xlsx", "README.md"]:
        file_path = os.path.join(output_dir, file_name)
        if os.path.exists(file_path):
            size_kb = os.path.getsize(file_path) / 1024
            files_created.append(f"  • {file_name}: {size_kb:.1f} KB")
    
    if files_created:
        for file_info in files_created:
            print(file_info)
    else:
        print("  ✗ Файлы не были созданы")
    
    print(f"\nКоличество записей: {len(df):,}".replace(",", " "))
    print(f"Количество столбцов: {len(df.columns)}")
    
    # Показываем первые 3 записи для проверки
    print(f"\nПервые 3 записи для проверки:")
    print("-" * 80)
    for i in range(min(3, len(df))):
        print(f"Запись #{i+1}:")
        print(f"  Бренд: {df.iloc[i].get('brand_name', 'N/A')}")
        print(f"  Название: {df.iloc[i].get('imt_name', 'N/A')[:50]}...")
        print(f"  Описание: {df.iloc[i].get('description', 'N/A')[:80]}...")
        print("-" * 80)
        
else:
    print("✗ Не удалось создать DataFrame. Проверьте данные.")

print("\n" + "=" * 50)
print("СОХРАНЕНИЕ ЗАВЕРШЕНО!")

СОХРАНЕНИЕ В PARQUET И CSV ФОРМАТЫ:
✓ Создана/проверена папка: d:\Projects\thesis\data\goods

Преобразую данные в DataFrame...
✓ DataFrame создан. Размер: 1000 строк × 9 столбцов

Столбцы в данных (9):
   1. imt_id
   2. nm_id
   3. imt_name
   4. subj_name
   5. subj_root_name
   6. nm_colors_names
   7. vendor_code
   8. description
   9. brand_name

------------------------------
Сохранение в Parquet формат...
✓ Parquet файл сохранен: d:\Projects\thesis\data\goods\goods_data.parquet
  Размер файла: 0.15 MB
  Сжатие: snappy
  Столбцы: 9

------------------------------
Сохранение в CSV формат...
✓ CSV файл сохранен: d:\Projects\thesis\data\goods\goods_data.csv
  Размер файла: 0.40 MB
  Кодировка: UTF-8
  Разделитель: запятая
✗ Ошибка при сохранении CSV: [WinError 2] Не удается найти указанный файл: 'd:\\Projects\\thesis\\data\\goods\\goods_data_tab.сsv'

------------------------------
Сохранение в Excel формат (опционально)...
✓ Excel файл сохранен: d:\Projects\thesis\data\goods\goods